In [18]:
# Importing necessary modules

import os
import math
import pandas as pd
import numpy as np
import glob
from itertools import cycle
import matplotlib.pyplot as plt
from scipy import stats
import shutil



# Identifying directory of lab sheets for quality screening

path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = "Lab_Sheets/"

datasheets = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder

glob.os.chdir(datasheets)



# Pulling data to navigate to completed folders in each topic folder

topic_folders = []

for x, folder, file in os.walk(datasheets):
    for loc in folder:
        topic_folders.append(loc)
        
topic_folders = [x for x in topic_folders if x != 'Completed']
topic_folders

for i in topic_folders:
    
    try:
    
        print(i)

        folder_name = str(i)
        
        location = "Student_Lists/Completed/"

        sheet = i + '.csv'

        id_file = '/Users/' + user + '/Desktop/Coding/StandardLab/' + location

        glob.os.chdir(id_file)

        u_id_list = pd.read_csv(sheet, header=None)

        u_id_list.columns = ['Sample_ID']

        ureides = u_id_list
        
        n_id_list = pd.read_csv(sheet, header=None)

        n_id_list.columns = ['Sample_ID']
        
        nitrates = n_id_list
        
        screened_data = datasheets + i + '/Completed'

        glob.os.chdir(screened_data)

        alldata = glob.glob(screened_data + "/*.csv")

        for loc in alldata:

            analysis = (loc.split('_')[-7])
            flag = (loc.split('_')[-11])
            name = (loc.split('\\')[-1])

            if flag != 'BAD':

                if analysis == "NIT":

                    var_id = []
                    nit_id = []
                    nit_conc = []

                    print(loc)
                    DF = pd.read_csv(loc)
                    #print(DF)

                    nit_df = DF[['Sample_ID', 'Absorbance']]

                    nit_g = nit_df.groupby('Sample_ID')

                    nit_final=nit_df.merge(nit_g.mean(),on='Sample_ID')

                    nit_final.rename(columns = {'Absorbance_x':'Each', 'Absorbance_y':'Mean'}, inplace=True)


                    # Marking values with high variance

                    sample_data_var = nit_df.merge((abs((nit_g.max() - nit_g.min())) / (nit_g.max())), on='Sample_ID')

                    sample_data_var['Type'] = DF['Type']

                    for i, row in sample_data_var.iterrows():

                        if row['Type'] == "O":

                            if float(row['Absorbance_y']) > 0.1:
                                var_id.append('x_')
                            else:
                                var_id.append('')

                    nit_final['Type'] = DF['Type']

                    nit_final['Sample_Wt(g)'] = DF['Sample_Wt(g)']

                    nit_final['Sample_Wt(g)'].replace(np.nan, 0.3, inplace= True)

                    # Creating empty lists to append with 'blanks' information

                    blanks_abs = []


                    # Identifying blank absorbances based on 'Type' column

                    for n in range(len(nit_final)):
                        if nit_final.Type[n] == 'B':
                            blanks_abs.append(float(nit_final.Each[n]))


                        # Creating a conditional to filter blank absorbances outside of acceptable range

                    good_blanks = [x for x in blanks_abs if x < 0.1 and x > -0.1]

                    blank_val = np.mean(good_blanks)

                    c=[]
                    e=[]

                    for i, row in nit_final.iterrows():

                         if row['Type'] == 'C':

                            c.append(float(row['Sample_ID']))
                            e.append(float(row['Each']))


                    slope, intercept, r_value, p_value, std_err = stats.linregress(e, c)

                    if r_value**2 < 0.95:

                            # Creating pandas dataframe for analysis

                            columns = [['Conc', 'Abs']]

                            curve = pd.DataFrame(columns = columns)


                            # Assigning values to concentration and absorbance columns

                            curve['Conc'] = c
                            curve['Abs'] = e


                            # Defining the size of the possible removal matrix

                            points = len(c) # Total number of datapoints


                            # Creating a matrix via a numpy array 

                            array = np.identity(points)


                            # Using a for-loop to generate new columns in the new pandas df containing values from decision matrix

                            for i in range(array.shape[1]):

                                curve[str(i)] = array[:,i]


                            # Appending 0's and 1's to df for all combinations of 1 and 2-point removal

                            for i in range(1,points):

                                array = np.identity(i)
                                array = np.vstack([array, [1]*i])

                                if array.shape[0] != 12:
                                    for j in range(12 - array.shape[0]):
                                        array = np.vstack([array, [0]*array.shape[1]])

                                    for k in range(array.shape[1]):

                                        curve[str(i)+str(k)+str(k)] = array[:,k]


                            # Assigning column names and creating pandas dataframe to hold/analyze "new" curve parameters

                            columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

                            columns = []
                            slopes = []
                            intercepts = []
                            rvals = []
                            pvals = []
                            stderrs = []

                            parameters = pd.DataFrame(columns = columns)


                            # Running conditional by column in new df (ONLY CONSIDERING THE COLUMNS OF THE MATRIX IN WHICH ONE POINT IS REMOVED)

                            for column in curve.iloc[:, 2:14]:

                                columns.append(column)


                                # Creating iterations counter and creating lists to append in for-loop

                                count = 0
                                conc = []
                                absorb = []


                                # Running conditional by row in column

                                for i in curve[column]: 


                                    # Conditional "removing" rows of column by not appending values to respective lists if value is 1

                                    if i < 1:
                                        conc.append(curve.iloc[count,0])
                                        absorb.append(curve.iloc[count,1])


                                    # Appending counter to reflect the next row for iteration

                                    count = count + 1


                                # Assigning variables to outputs from stats.linregress() function and appending list with column values

                                slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)

                                slopes.append(slope)
                                intercepts.append(inter)
                                rvals.append(r_val**2)
                                pvals.append(p_val)
                                stderrs.append(std)


                            # Assigning regression values to columns in parameters dataframe    

                            parameters['Column'] = columns
                            parameters['Slope'] = slopes
                            parameters['Intercept'] = intercepts
                            parameters['R_Sq'] = rvals
                            parameters['P_Val'] = pvals
                            parameters['StdErr'] = stderrs


                            ## TO SEE THE PARAMETERS AND PRINT THE R^2 CALCULATED BY REMOVAL OF ONE POINT OF THE CURVE:

                            #print(parameters['R_Sq'].max())
                            #print(parameters)


                            # Creating a second conditional for r^2 based on a matrix that calculates optional r^2 values based
                            # on removal of two curve points ... THIS CODE WILL ONLY BE RUN IF THE HIGHEST POSSIBLE R^2 VALUE IN
                            # THE PREVIOUS DATAFRAME IS LESS THAN 0.95

                            if parameters['R_Sq'].max() < 0.95:


                                # OVERWRITING previous column names and dataframe to hold/analyze "new" secondary curve parameters

                                columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

                                columns = []
                                slopes = []
                                intercepts = []
                                rvals = []
                                pvals = []
                                stderrs = []

                                parameters = pd.DataFrame(columns = columns)


                                # Running conditional by column in new df (ONLY CONSIDERING THE COLUMNS OF THE MATRIX IN WHICH TWO POINTS ARE REMOVED)

                                for column in curve.iloc[:, 14:]:

                                    columns.append(column)


                                    # Creating iterations counter and creating lists to append in for-loop

                                    count = 0
                                    conc = []
                                    absorb = []


                                    # Running conditional by row in column

                                    for i in curve[column]: 


                                        # Conditional "removing" rows of column by not appending values to respective lists if value is 1

                                        if i < 1:
                                            conc.append(curve.iloc[count,0])
                                            absorb.append(curve.iloc[count,1])


                                        # Appending counter to reflect the next row for iteration    

                                        count = count + 1


                                    # Assigning variables to outputs from stats.linregress() function and appending list with column values    

                                    slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)

                                    slopes.append(slope)
                                    intercepts.append(inter)
                                    rvals.append(r_val**2)
                                    pvals.append(p_val)
                                    stderrs.append(std)

                                # Assigning regression values to columns in parameters dataframe 

                                parameters['Column'] = columns
                                parameters['Slope'] = slopes
                                parameters['Intercept'] = intercepts
                                parameters['R_Sq'] = rvals
                                parameters['P_Val'] = pvals
                                parameters['StdErr'] = stderrs


                            # Displaying FINAL optimized r^2 value ... either from removing one or two points (df overwritten if two points)

                            #print(parameters.R_Sq.max())

                    nconc = []
                    nit_conc_var = []


                    for i, row in nit_final.iterrows():

                        nconc.append(((row['Mean'] - blank_val)*slope + intercept)/((row['Sample_Wt(g)']*1000/7.5)))

                    nit_final['Sample_Conc(micrmol/g)'] = nconc

                    for i, row in nit_final.iterrows():

                         if row['Type'] == 'O':

                            nit_id.append(row['Sample_ID'])
                            nit_conc.append(row['Sample_Conc(micrmol/g)'])


                    for i in range(len(nit_conc)):
                        nit_conc_var.append(str(var_id[i]) + str(nit_conc[i]))       


                    ncolumns = ['Sample_ID', 'Conc']

                    ndata = pd.DataFrame(columns = ncolumns)

                    ndata['Sample_ID'] = nit_id
                    ndata['Conc'] = nit_conc_var

                    final_nitrates = pd.merge(nitrates, ndata, on='Sample_ID', how = 'left')

                    complete_nitrates = nitrates

                    complete_nitrates[str(name)] = final_nitrates['Conc']

        
        
        for loc in alldata:

            analysis = (loc.split('_')[-7])
            flag = (loc.split('_')[-11])
            name = (loc.split('\\')[-1])

            if flag != 'BAD':
        
                if analysis == "URE":
        
                    var_id = []
                    ure_id = []
                    ure_conc = []

                    print(loc)
                    DF = pd.read_csv(loc)
                    #print(DF)

                    ure_df = DF[['Sample_ID', 'Absorbance']]

                    ure_g = ure_df.groupby('Sample_ID')

                    ure_final = ure_df.merge(ure_g.mean(),on='Sample_ID')

                    ure_final.rename(columns = {'Absorbance_x':'Each', 'Absorbance_y':'Mean'}, inplace=True)



                    # Marking values with high variance

                    sample_data_var = ure_df.merge((abs((ure_g.max() - ure_g.min())) / (ure_g.max())), on='Sample_ID')

                    sample_data_var['Type'] = DF['Type']

                    for i, row in sample_data_var.iterrows():

                        if row['Type'] == "O":

                            if float(row['Absorbance_y']) > 0.1:
                                var_id.append('x_')
                            else:
                                var_id.append('')
                  
                    ure_final['Type'] = DF['Type']

                    ure_final['Sample_Wt(g)'] = DF['Sample_Wt(g)']

                    ure_final['Sample_Wt(g)'].replace(np.nan, 0.3, inplace= True)


                    # Creating empty lists to append with 'blanks' information

                    blanks_abs = []


                    # Identifying blank absorbances based on 'Type' column

                    for n in range(len(ure_final)):
                        if ure_final.Type[n] == 'B':
                            blanks_abs.append(float(ure_final.Each[n]))


                        # Creating a conditional to filter blank absorbances outside of acceptable range

                    good_blanks = [x for x in blanks_abs if x < 0.1 and x > -0.1]

                    blank_val = np.mean(good_blanks)

                    c=[]
                    e=[]

                    for i, row in ure_final.iterrows():

                         if row['Type'] == 'C':

                            c.append(float(row['Sample_ID']))
                            e.append(float(row['Each']))


                    slope, intercept, r_value, p_value, std_err = stats.linregress(e, c)

                    if r_value**2 < 0.95:

                        # Creating pandas dataframe for analysis

                        columns = [['Conc', 'Abs']]

                        curve = pd.DataFrame(columns = columns)


                        # Assigning values to concentration and absorbance columns

                        curve['Conc'] = c
                        curve['Abs'] = e


                        # Defining the size of the possible removal matrix

                        points = len(c) # Total number of datapoints


                        # Creating a matrix via a numpy array 

                        array = np.identity(points)


                        # Using a for-loop to generate new columns in the new pandas df containing values from decision matrix

                        for i in range(array.shape[1]):

                            curve[str(i)] = array[:,i]


                        # Appending 0's and 1's to df for all combinations of 1 and 2-point removal

                        for i in range(1,points):

                            array = np.identity(i)
                            array = np.vstack([array, [1]*i])

                            if array.shape[0] != 12:
                                for j in range(12 - array.shape[0]):
                                    array = np.vstack([array, [0]*array.shape[1]])

                                for k in range(array.shape[1]):

                                    curve[str(i)+str(k)+str(k)] = array[:,k]


                        # Assigning column names and creating pandas dataframe to hold/analyze "new" curve parameters

                        columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

                        columns = []
                        slopes = []
                        intercepts = []
                        rvals = []
                        pvals = []
                        stderrs = []

                        parameters = pd.DataFrame(columns = columns)


                        # Running conditional by column in new df (ONLY CONSIDERING THE COLUMNS OF THE MATRIX IN WHICH ONE POINT IS REMOVED)

                        for column in curve.iloc[:, 2:14]:

                            columns.append(column)


                            # Creating iterations counter and creating lists to append in for-loop

                            count = 0
                            conc = []
                            absorb = []


                            # Running conditional by row in column

                            for i in curve[column]: 


                                # Conditional "removing" rows of column by not appending values to respective lists if value is 1

                                if i < 1:
                                    conc.append(curve.iloc[count,0])
                                    absorb.append(curve.iloc[count,1])


                                # Appending counter to reflect the next row for iteration

                                count = count + 1


                            # Assigning variables to outputs from stats.linregress() function and appending list with column values

                            slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)

                            slopes.append(slope)
                            intercepts.append(inter)
                            rvals.append(r_val**2)
                            pvals.append(p_val)
                            stderrs.append(std)


                        # Assigning regression values to columns in parameters dataframe    

                        parameters['Column'] = columns
                        parameters['Slope'] = slopes
                        parameters['Intercept'] = intercepts
                        parameters['R_Sq'] = rvals
                        parameters['P_Val'] = pvals
                        parameters['StdErr'] = stderrs


                        ## TO SEE THE PARAMETERS AND PRINT THE R^2 CALCULATED BY REMOVAL OF ONE POINT OF THE CURVE:

                        #print(parameters['R_Sq'].max())
                        #print(parameters)


                        # Creating a second conditional for r^2 based on a matrix that calculates optional r^2 values based
                        # on removal of two curve points ... THIS CODE WILL ONLY BE RUN IF THE HIGHEST POSSIBLE R^2 VALUE IN
                        # THE PREVIOUS DATAFRAME IS LESS THAN 0.95

                        if parameters['R_Sq'].max() < 0.95:


                            # OVERWRITING previous column names and dataframe to hold/analyze "new" secondary curve parameters

                            columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

                            columns = []
                            slopes = []
                            intercepts = []
                            rvals = []
                            pvals = []
                            stderrs = []

                            parameters = pd.DataFrame(columns = columns)


                            # Running conditional by column in new df (ONLY CONSIDERING THE COLUMNS OF THE MATRIX IN WHICH TWO POINTS ARE REMOVED)

                            for column in curve.iloc[:, 14:]:

                                columns.append(column)


                                # Creating iterations counter and creating lists to append in for-loop

                                count = 0
                                conc = []
                                absorb = []


                                # Running conditional by row in column

                                for i in curve[column]: 


                                    # Conditional "removing" rows of column by not appending values to respective lists if value is 1

                                    if i < 1:
                                        conc.append(curve.iloc[count,0])
                                        absorb.append(curve.iloc[count,1])


                                    # Appending counter to reflect the next row for iteration    

                                    count = count + 1


                                # Assigning variables to outputs from stats.linregress() function and appending list with column values    

                                slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)

                                slopes.append(slope)
                                intercepts.append(inter)
                                rvals.append(r_val**2)
                                pvals.append(p_val)
                                stderrs.append(std)

                            # Assigning regression values to columns in parameters dataframe 

                            parameters['Column'] = columns
                            parameters['Slope'] = slopes
                            parameters['Intercept'] = intercepts
                            parameters['R_Sq'] = rvals
                            parameters['P_Val'] = pvals
                            parameters['StdErr'] = stderrs


                        # Displaying FINAL optimized r^2 value ... either from removing one or two points (df overwritten if two points)

                        #print(parameters.R_Sq.max())

                    conc = []
                    ure_conc_var = []

                    for i, row in ure_final.iterrows():

                        conc.append((((row['Mean'] - blank_val)*slope + intercept)/((row['Sample_Wt(g)']*1000/7.5)))*4)

                    ure_final['Sample_Conc(micrmol/g)'] = conc



                    for i, row in ure_final.iterrows():

                        if row['Type'] == 'O':

                            ure_id.append(row['Sample_ID'])
                            ure_conc.append(row['Sample_Conc(micrmol/g)'])

                    for i in range(len(ure_conc)):
                        ure_conc_var.append(str(var_id[i]) + str(ure_conc[i]))

                    columns = ['Sample_ID', 'Conc']

                    udata = pd.DataFrame(columns = columns)

                    udata['Sample_ID'] = ure_id
                    udata['Conc'] = ure_conc_var 

                    final_ureides = pd.merge(ureides, udata, on='Sample_ID', how = 'left')  

                    complete_ureides = ureides

                    complete_ureides[str(name)] = final_ureides['Conc']
                
        save_loc = '/Users/' + user + '/Desktop/Coding/StandardLab/Results/'

        glob.os.chdir(save_loc)
        
        complete_ureides.to_csv('Results_URE_' + folder_name + '.csv', index=False)
        complete_nitrates.to_csv('Results_NIT_' + folder_name + '.csv', index=False)
        
        
        ## BUG AS OF 4-12-19 .... BOTH FILES SAVED ARE IDENTICAL ... SOMEHOW FILES ARE BEING OVREWRITTEN? PROBABLY A NESTING ISSUE.
        
        
    
    except FileNotFoundError:
        continue

MS_ASH_LI
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/MS_ASH_LI/Completed\A_1_NIT_MS_ASH_LI_12_17_18.csv
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/MS_ASH_LI/Completed\A_2_NIT_MS_ASH_LI_12_17_18.csv
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/MS_ASH_LI/Completed\A_3_NIT_MS_ASH_LI_12_18_18.csv
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/MS_ASH_LI/Completed\A_4_NIT_MS_ASH_LI_12_18_18.csv
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/MS_ASH_LI/Completed\A_5_NIT_MS_ASH_LI_12_19_18.csv
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/MS_ASH_LI/Completed\A_6_NIT_MS_ASH_LI_00_00_00.csv
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/MS_ASH_LI/Completed\A_8_NIT_MS_ASH_LI_12_20_18.csv
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/MS_ASH_LI/Completed\B_2_NIT_MS_ASH_LI_01_03_18.csv
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/MS_ASH_LI/Completed\B_3_NIT_MS_ASH_LI_01_04_19.csv
/Users/rveenstra/Desktop/Coding/Stan

In [86]:
# Importing necessary modules

import os
import math
import pandas as pd
import numpy as np
import glob
from itertools import cycle
import matplotlib.pyplot as plt
from scipy import stats
import shutil



# Identifying directory of lab sheets for quality screening

path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = "Lab_Sheets/"

datasheets = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder

glob.os.chdir(datasheets)



# Setting loop for iterating through files WITHIN folders IN the specified folder

for x, folder, file in os.walk(datasheets):
    
    
    # Using "try" loop in the event that a folder is empty
    
    try: 
        
        
        # Looping through every file in a folder
        
        for loc in folder:
            
            
            # Identifying working folder
            
            print(loc)

            folder_loc = datasheets + loc

            glob.os.chdir(folder_loc)
            
            
            # Grabbing all csv files in specified folder

            all_datasheets = glob.glob(folder_loc + "/*.csv")
            
            
            # Creating empty lists to append with ID's that fail quality screening

            N_Re_Run = []
            U_Re_Run = []
            
            
            # Screening each sheet individually

            for sheet in all_datasheets:

                
                # Reading working file as dataframe in pandas
                
                lab_data = pd.read_csv(sheet)
                
                
                # Pulling parts of name to identify sheet for naming/moving

                sheet_name = sheet.split('\\')[-1]
                title = sheet_name.split('_')[2]
            
            
                # Checking for data - skipping remainder of code (breaking loop) if there is no data

                if np.isnan(lab_data.Absorbance[0]):
                    break

                
                # Continuing if data is found

                else:
                    
                    # Identifying sheet
                    
                    print(sheet)


                    # Creating empty lists to append with 'blanks' information
                    
                    blanks_abs = []

                    
                    # Identifying blank absorbances based on 'Type' column
                    
                    for n in range(len(lab_data)):
                        if lab_data.Type[n] == 'B':
                            blanks_abs.append(float(lab_data.Absorbance[n]))
                            
                  
                    # Creating a conditional to filter blank absorbances outside of acceptable range
                            
                    good_blanks = [x for x in blanks_abs if x < 0.1 and x > -0.1]
                    
                    
                    # If the list of acceptable blanks has fewer than three values, set must be repeated
                    if len(good_blanks) <= 2 :
                        print('oops')
                    else:
                        continue
    except:
        continue

MR_USB
RV_CTS
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/RV_CTS\A_1_URE_RV_CTS_00_00_00.csv
oops
USB_Indiana
USB_Iowa
USB_Minnesota
USB_South_Dakota
Completed
Completed
Completed
Completed
Completed
Completed


In [36]:
columns = ['Conc', 'Abs']

curve = pd.DataFrame(columns = columns)

curve['Conc'] = curve_concentration
curve['Abs'] = curve_absorbance

points = len(curve_concentration) # Total number

array = np.identity(points)

for i in range(array.shape[1]):

    curve[str(i)] = array[:,i]

for i in range(1,points):

    array = np.identity(i)
    array = np.vstack([array, [1]*i])

    if array.shape[0] != 12:
        for j in range(12 - array.shape[0]):
            array = np.vstack([array, [0]*array.shape[1]])

        for k in range(array.shape[1]):

            curve[str(i)+str(k)+str(k)] = array[:,k]
columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

columns = []
slopes = []
intercepts = []
rvals = []
pvals = []
stderrs = []

parameters = pd.DataFrame(columns = columns)

for column in curve.iloc[:, 2:14]:
    
    columns.append(column)
    
    count = 0
    conc = []
    absorb = []
    
    
    for i in curve[column]: 
        if i < 1:
            conc.append(curve.iloc[count,0])
            absorb.append(curve.iloc[count,1])
        count = count + 1
        
    slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)
    
    slopes.append(slope)
    intercepts.append(inter)
    rvals.append(r_val**2)
    pvals.append(p_val)
    stderrs.append(std)

parameters['Column'] = columns
parameters['Slope'] = slopes
parameters['Intercept'] = intercepts
parameters['R_Sq'] = rvals
parameters['P_Val'] = pvals
parameters['StdErr'] = stderrs

print(parameters.R_Sq.max())

if parameters['R_Sq'].max() < 0.90:
    
    columns = ['Column', 'Slope', 'Intercept', 'R_Val', 'P_Val', 'StdErr']

    columns = []
    slopes = []
    intercepts = []
    rvals = []
    pvals = []
    stderrs = []

    parameters = pd.DataFrame(columns = columns)

    for column in curve.iloc[:, 14:]:

        columns.append(column)
        
        count = 0
        conc = []
        absorb = []


        for i in curve[column]: 
            if i < 1:
                conc.append(curve.iloc[count,0])
                absorb.append(curve.iloc[count,1])
            count = count + 1

        slope, inter, r_val, p_val, std = stats.linregress(conc, absorb)

        slopes.append(slope)
        intercepts.append(inter)
        rvals.append(r_val**2)
        pvals.append(p_val)
        stderrs.append(std)
    #     print(conc)
    #     print(r_val**2)
    parameters['Column'] = columns
    parameters['Slope'] = slopes
    parameters['Intercept'] = intercepts
    parameters['R_Sq'] = rvals
    parameters['P_Val'] = pvals
    parameters['StdErr'] = stderrs
    
parameters

0.8587038805948787


,Column,Slope,Intercept,R_Sq,P_Val,StdErr
0,100,0.000354,-0.245000,0.581206,0.010353,0.000106
1,200,0.000342,-0.201250,0.556845,0.013184,0.000108
2,211,0.000354,-0.245000,0.581206,0.010353,0.000106
3,300,0.000319,-0.110909,0.578776,0.010612,0.000096
4,311,0.000328,-0.144793,0.597123,0.008779,0.000095
5,322,0.000319,-0.110909,0.578776,0.010612,0.000096
6,400,0.000320,-0.114876,0.579201,0.010566,0.000096
7,411,0.000329,-0.148760,0.597592,0.008736,0.000095
8,422,0.000320,-0.114876,0.579201,0.010566,0.000096
9,433,0.000306,-0.055556,0.604321,0.008133,0.000088
